In [1]:
include("nnls.jl")

Test Summary:          |  Pass  Total
  construct_householder! | 20000  20000
Test Summary:      |  Pass  Total
  apply_householder! | 30000  30000
Test Summary:     |  Pass  Total
  orthogonal_rotmat | 10000  10000
main loop
main loop
 main loop
 main loop
work.mode = Base.RefValue{Int32}(1)
nnls: Test Failed
  Expression: A * work1.x ≈ b
   Evaluated: [0.510033,-0.57948,-0.0256952] isapprox [1.15156,-0.0387917,0.514559]
 in record(::Base.Test.DefaultTestSet, ::Base.Test.Fail) at ./test.jl:431
 in do_test(::Base.Test.Returned, ::Expr) at ./test.jl:281
 in macro expansion; at /Users/rdeits/locomotion/explorations/drake-mpc/notebooks/nnls.jl:504 [inlined]
 in macro expansion; at ./test.jl:674 [inlined]
 in anonymous at ./<missing>:?
 in include_from_node1(::String) at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 in include_from_node1(::String) at /Applications/Julia-0.5.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 in include_string(::String, ::Strin

LoadError: LoadError: Some tests did not pass: 66 passed, 34 failed, 0 errored, 0 broken.
while loading /Users/rdeits/locomotion/explorations/drake-mpc/notebooks/nnls.jl, in expression starting on line 485

In [1]:
macro wrappedallocs(expr)
    argnames = [gensym() for a in expr.args]
    quote
        function g($(argnames...))
            @allocated $(Expr(expr.head, argnames...))
        end
        $(Expr(:call, :g, [esc(a) for a in expr.args]...))
    end
end

@wrappedallocs (macro with 1 method)

In [3]:
# SUBROUTINE H12 (MODE,LPIVOT,L1,M,U,IUE,UP,C,ICE,ICV,NCV) 

In [33]:
u = normalize([1, 1])
lpivot = 1
up = construct_householder!(lpivot, u)
@show u up
c = [1., 0]
apply_householder!(lpivot, u, up, c)
c

u = [-1.0,0.707107]
up = 1.7071067811865475


2-element Array{Float64,1}:
 -0.707107
 -0.707107

In [30]:
v = normalize([1., 1])

2-element Array{Float64,1}:
 0.707107
 0.707107

In [24]:
x = [1., 0]

2-element Array{Float64,1}:
 1.0
 0.0

In [25]:
x - 2 * v * v' * x

2-element Array{Float64,1}:
  2.22045e-16
 -1.0        

In [28]:
u

2-element Array{Float64,1}:
 -1.0     
  0.707107

In [27]:
c = [1., 0]
apply_householder!(lpivot, u, up, c)
c

2-element Array{Float64,1}:
 -0.707107
 -0.707107

In [19]:
function h12{T}(mode::Integer, lpivot::Integer, l1::Integer, m::Integer, u::Matrix{T}, up::T, C::Matrix{T})
    @assert size(u, 2) == m
    ncv = size(C, 2)
    icv = size(C, 1)
    ice = 1
    
    if 0 >= lpivot || lpivot >= l1 || l1 > m
        return
    end
    
    cl = abs(u[1, lpivot])
    
    if mode == 2
        if cl <= 0
            return
        end
    else
        # ****** CONSTRUCT THE TRANSFORMATION. ******
        for j in l1:m
            cl = max(abs(u[1, j]), cl)
        end
        
        if cl <= 0
            return
        end
        
        clinv = 1 / cl
        sm = (u[1, lpivot] * clinv)^2
        for j in l1:m
            sm += (u[1, j] * clinv)^2
        end
        cl *= √sm
        if u[1, lpivot] > 0
            cl = -cl
        end
        up = u[1, lpivot] - cl
        u[1, lpivot] = cl
    end
    
    # ****** APPLY THE TRANSFORMATION  I+U*(U**T)/B  TO C. ******
    if ncv <= 0
        return
    end
    
    b = up * u[1, lpivot]
    
    # B  MUST BE NONPOSITIVE HERE.  IF B = 0., RETURN.
    if b >= 0
        return
    end
    
    b = 1 / b
    ic2 = 1 - icv + ice * (lpivot - 1)
    incr = ice * (l1 - lpivot)
    for j in 1:ncv
        i2 += icv
        i3 = i2 + incr
        i4 = i3
        sm = c[i2] * up
        for i in l1:m
            sm += c[i3] * u[1, i]
            i3 += ice
        end
        if sm != 0
            sm *= b
            c[i2] = c[i2] + sm * up
            for i in l1:m
                c[i4] += sm * u[1, i]
                i4 += ice
            end
        end
    end                
end

h12 (generic function with 1 method)

In [17]:


function nnls!{T}(work::NNLSWorkspace{T}, A::Matrix{T}, b::Vector{T}, itermax=(3 * size(A, 2)))
    x = work.x
    rnorm = work.rnorm
    w = work.w
    zz = work.zz
    idx = work.idx
    
    m, n = size(A)
    @assert size(b) == (m,)
    @assert size(x) == (n,)
    @assert size(rnorm) == (m,)
    @assert size(w) == (n,)
    @assert size(zz) == (m,)
    @assert size(idx) == (n,)
    
    iter = 0
    x .= 0
    idx .= 1:n
    
    izmax = 0
    iz2 = n
    iz1 = 1
    nsetp = 0
    npp1 = 1
    
    @label l30
    
    while true
        # QUIT IF ALL COEFFICIENTS ARE ALREADY IN THE SOLUTION.
        # OR IF M COLS OF A HAVE BEEN TRIANGULARIZED. 
        if (iz1 > iz2 || nsetp >= m)
            break
        end
        
        # COMPUTE COMPONENTS OF THE DUAL (NEGATIVE GRADIENT) VECTOR W().
        for iz in iz1:iz2
            j = idx[iz]
            sm = 0
            for l in npp1:m
                sm += A[l, j] * b[l]
            end
            w[j] = sm
        end
        
        # FIND LARGEST POSITIVE W(J).
        wmax = 0
        for iz in iz1:iz2
            j = idx[iz]
            if w[j] > wmax
                wmax = w[j]
                izmax = iz
            end
        end
        
        # IF WMAX .LE. 0. GO TO TERMINATION.
        # THIS INDICATES SATISFACTION OF THE KUHN-TUCKER CONDITIONS.
        if wmax <= 0
            break
        end
        
        iz = izmax
        j = idx[iz]
        
        # THE SIGN OF W(J) IS OK FOR J TO BE MOVED TO SET P.
        # BEGIN THE TRANSFORMATION AND CHECK NEW DIAGONAL ELEMENT TO AVOID
        # NEAR LINEAR DEPENDENCE.
        Asave = A[npp1, j]
        
    end
        
    
end


nnls! (generic function with 4 methods)

In [18]:
n = 5
m = 6
A = rand(m, n)
b = rand(m)
x = Vector{Float64}(n)
rnorm = Vector{Float64}(m)
w = Vector{Float64}(n)
zz = Vector{Float64}(m)
idx = Vector{Int}(n)
work = NNLSWorkspace(x, rnorm, w, zz, idx)

nnls!(work, A, b)
@assert @wrappedallocs(nnls!(work, A, b)) == 0
nnls!(work, A, b)

LoadError: InterruptException: